In [1]:
import numpy as np
import setigen as stg
from blimpy import Waterfall
import matplotlib.pyplot as plt
import random
import os
from astropy import units as u
from tqdm import tqdm
from sklearn.metrics import silhouette_score
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

os.environ["CUDA_VISIBLE_DEVICES"]="0"



2023-05-28 17:25:23.049064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def painting(data):
    all_data = []
    labels = []
    for c in range(num_classes):
        drift = 2*random.random()*(-1)**random.randint(0,2)
        snr = random.randint(100, 150)
        width = random.randint(20, 50)
        for s in range(num_samples_per_class):
            index = random.randint(0, data.shape[0]-1)
            window = data[index, :,:]
            
            start = random.randint(50, 180)
            
            frame = stg.Frame.from_data(df=2.7939677238464355*u.Hz,
                                        dt=18.253611008*u.s,
                                        fch1=1289*u.MHz,
                                        ascending=True,
                                        data=window)
            frame.add_signal(stg.constant_path(
                                        f_start=frame.get_frequency(index=start),
                                       drift_rate=drift*u.Hz/u.s),
                                      stg.constant_t_profile(level=frame.get_intensity(snr=snr)),
                                      stg.gaussian_f_profile(width=width*u.Hz),
                                      stg.constant_bp_profile(level=1))
            all_data.append(frame.data)
            labels.append(c)
    all_data = np.array(all_data)
    labels = np.vstack(labels)
    return all_data, labels

In [3]:
import cv2
import numpy as np

def transform(data):
    final = []
    for i in range(data.shape[0]):
        img = np.repeat(data[i,:, :], 3, axis=2)
        res = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        final.append(res)
    return np.array(final)


In [4]:
from tqdm import tqdm
import gc

def normalize(data):
    epsilon = 1
    min_val = data.min()
    data = data - min_val + epsilon
    new_data = np.log(data)
    min_val = data.min()
    max_val = data.max()
    final_data = (data - min_val) / (max_val - min_val)
    return final_data
    
def normalize_data(data):
    for i in range(data.shape[0]):
        data[i,:,:] = normalize(data[i,:,:, :])
    return data

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Model

model = ResNet50(include_top=True,
                weights='imagenet',
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=1000
                )
truncated_model = Model(inputs = model.layers[0].input, outputs = model.layers[-1].output)
truncated_model.summary()
truncated_model.compile(optimizer='rmsprop')



2023-05-28 17:25:29.507206: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 17:25:29.866745: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14233 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [5]:
def idealized(num=10_000):
    drift = 0
    snr = 20
    width = 10
    start = 256//2
    data = []
    labels = []
    for tag in range(4):
        label_vec = np.zeros(4)
        label_vec[tag] = 1
        for i in range(num):
            if tag == 0:
                drift = (-1)**(random.randint(0,2)) * 2 *random.random()
            elif tag == 1:
                snr = 50*random.random() +20
            elif tag == 2:
                width = 50*random.random() +20
            elif tag == 3:
                start = random.randint(50, 180)

            frame = stg.Frame(fchans=256*u.pixel,
                              tchans=16*u.pixel,
                              df=2.7939677238464355*u.Hz,
                              dt=18.253611008*u.s,
                              fch1=6095.214842353016*u.MHz)
            noise = frame.add_noise(x_mean=10, noise_type='chi2')
            frame.add_signal(stg.constant_path(
                                        f_start=frame.get_frequency(index=start),
                                       drift_rate=drift*u.Hz/u.s),
                                      stg.constant_t_profile(level=frame.get_intensity(snr=snr)),
                                      stg.gaussian_f_profile(width=width*u.Hz),
                                      stg.constant_bp_profile(level=1))
            data.append(frame.data)
            labels.append(label_vec)
    data = np.array(data)
    labels = np.vstack(labels)
    return data, labels

In [6]:
import os
from tqdm import tqdm
import keras
total_scores = []
for i in tqdm(range(10)):
    idealized_signals, labels = idealized(num = 10_000)

    input_data = transform(np.expand_dims(idealized_signals, axis = -1))
    gc.collect()
    input_data = normalize_data(input_data)
    print(input_data[0,:,:].max(), input_data[0,:,:].min())
    splits = input_data.shape[0]//1_000
    X = []
    for i in range(splits):
        tensor = tf.convert_to_tensor(input_data[i*1_000:(i+1)*1_000, :,:,:], dtype=tf.float32)
        X.append(truncated_model(tensor))
    X = np.vstack(X)
    print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, random_state=42)

    linear = keras.Sequential(name="my_sequential")
    # linear.add(layers.Dense(1000, activation="linear", name="layer1"))
    linear.add(layers.Dense(4, activation="softmax", name="layer2"))

    linear.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    linear.fit(X_train, y_train, epochs=150, batch_size=128, validation_split = 0.1, shuffle=True)
    results = linear.evaluate(X_test, y_test)
    print(results)
    total_scores.append(results[-1])

  0%|                                                    | 0/10 [00:00<?, ?it/s]

1.0 0.0


2023-05-28 17:28:02.460324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-05-28 17:28:03.516960: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.79GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-28 17:28:03.517028: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.88GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-28 17:28:03.642134: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.79GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

(40000, 1000)
Epoch 1/150


2023-05-28 17:29:09.318031: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f48c80172f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-28 17:29:09.318141: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2023-05-28 17:29:09.325746: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-28 17:29:09.510158: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

[1.3066309690475464, 0.601893961429596]
1.0 0.0
(40000, 1000)
Epoch 1/150
189/189 [==============================] - 1s 2ms/step - loss: 1.3848 - accuracy: 0.2706 - val_loss: 1.3832 - val_accuracy: 0.2425
Epoch 2/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3813 - accuracy: 0.3620 - val_loss: 1.3795 - val_accuracy: 0.2425
Epoch 3/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3778 - accuracy: 0.3794 - val_loss: 1.3761 - val_accuracy: 0.3854
Epoch 4/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3743 - accuracy: 0.4498 - val_loss: 1.3723 - val_accuracy: 0.7134
Epoch 5/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3708 - accuracy: 0.6005 - val_loss: 1.3694 - val_accuracy: 0.4996
Epoch 6/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3673 - accuracy: 0.5235 - val_loss: 1.3656 - val_accuracy: 0.6806
Epoch 7/150
189/189 [==============================] - 0s 2ms/step - loss: 1.363

 90%|██████████████████████████████████████▋    | 9/10 [41:57<04:43, 283.40s/it]

[1.0034871101379395, 0.8225757479667664]
1.0 0.0
(40000, 1000)
Epoch 1/150
189/189 [==============================] - 1s 2ms/step - loss: 1.3843 - accuracy: 0.3715 - val_loss: 1.3832 - val_accuracy: 0.2425
Epoch 2/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3809 - accuracy: 0.3655 - val_loss: 1.3796 - val_accuracy: 0.2425
Epoch 3/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3776 - accuracy: 0.3728 - val_loss: 1.3766 - val_accuracy: 0.4750
Epoch 4/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3743 - accuracy: 0.4772 - val_loss: 1.3731 - val_accuracy: 0.4750
Epoch 5/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3711 - accuracy: 0.5163 - val_loss: 1.3697 - val_accuracy: 0.5146
Epoch 6/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3678 - accuracy: 0.5393 - val_loss: 1.3664 - val_accuracy: 0.7022
Epoch 7/150
189/189 [==============================] - 0s 2ms/step - loss: 1.36

100%|██████████████████████████████████████████| 10/10 [46:38<00:00, 279.89s/it]

[1.0276236534118652, 0.7695454359054565]


In [7]:
print(np.mean(total_scores))
print(np.std(total_scores))

0.741689395904541
0.06987631938237779
